# Prerequisites

In [1]:
import os, sys

import numpy as np
import tqdm
import torch

/home/marco/miniconda3/envs/nlp2022-hw3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Important paths for the notebook:

In [2]:
app_root = '../../../../'
test_name = 'test1'
datasets_path = os.path.join(app_root, 'data')
model_dir_path = os.path.join(app_root, 'model', test_name)

data_train_path = os.path.join(datasets_path, 'train.tsv')
data_dev_path = os.path.join(datasets_path, 'dev.tsv')

In [3]:
sys.path.append('../../../')

%load_ext autoreload
%autoreload 2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

Setting the seed for reproducibility:

In [4]:
SEED = 28

# random.seed(SEED) # not used
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [5]:
global_params = {
    'batch_size': 32,
    'PAD_TOKEN': '<pad>',
    'UNK_TOKEN': '<unk>',
    'transformer_name': "bert-base-cased",
    'mention_tags': {
        'p_open':  '<P>', 'p_close':  '</P>', 
        'e_open':  '<E>', 'e_close':  '</E>', 
    },
}

# Dataset

In [6]:
from stud.modelsTests.dataset.Dataset_transformer_simple import Dataset_transformer_simple

In [7]:
dataset_train = Dataset_transformer_simple(
    data_train_path, 
    tokenizer = global_params['transformer_name'],
    split_by_entities = True)

dataset_dev = Dataset_transformer_simple(
    data_dev_path, 
    tokenizer = global_params['transformer_name'],
    split_by_entities = True)

In [8]:
dataset_train.data_raw[14]

{'id': 'train-15',
 'text': "Twenty years ago, Lorenzo Uribe discovered true love with Maria Herrera and began a romance. Lorenzo was rich, married, and had a young son: Lautaro. Maria was poor and unknown to Lorenzo, had a daughter called Renata. Maria's mother, Gracia, wanted her daughter to catch this rich man at all costs and convinced her that pregnancy would assure this.",
 'pron': 'her',
 'p_offset': 250,
 'entity_A': 'Maria',
 'offset_A': 219,
 'is_coref_A': 'TRUE',
 'entity_B': 'Gracia',
 'offset_B': 235,
 'is_coref_B': 'FALSE'}

In [9]:
for i,e in enumerate(dataset_train.data_raw):
    if e['is_coref_A'] == e['is_coref_B'] and e['is_coref_A'] == 'FALSE':
        print(i)

6
15
17
28
31
33
45
46
49
55
67
78
80
88
92
97
99
101
112
114
121
122
141
155
178
180
190
200
223
233
239
244
254
256
279
334
335
340
348
349
350
357
368
374
376
394
399
404
406
412
417
426
428
438
452
486
500
504
513
528
529
531
534
540
557
564
581
632
656
661
668
675
687
720
732
733
745
784
815
833
834
838
847
872
873
882
885
900
905
915
917
923
926
936
941
968
982
984
988
996
998
1005
1006
1012
1043
1046
1088
1103
1109
1130
1146
1151
1160
1163
1170
1173
1180
1182
1183
1194
1230
1235
1237
1239
1243
1248
1251
1252
1259
1260
1261
1264
1270
1272
1282
1289
1290
1298
1304
1307
1331
1374
1390
1396
1408
1426
1461
1470
1499
1506
1531
1546
1559
1566
1576
1581
1582
1607
1618
1619
1625
1629
1631
1633
1638
1641
1643
1682
1694
1702
1703
1729
1744
1746
1750
1752
1753
1755
1776
1810
1818
1828
1836
1840
1856
1862
1869
1911
1919
1921
1925
1931
1937
1948
1949
1951
1961
1983
1985
1986
1990
2011
2028
2041
2049
2071
2082
2099
2106
2116
2120
2122
2123
2128
2134
2154
2156
2182
2186
2187
2190
2192
2196
2226

In [10]:
# Testing both entities as FALSE (pass (None, None)):

i_test = 6

ex_sample = (
    (dataset_train.data_raw[i_test]['pron'], dataset_train.data_raw[i_test]['p_offset']),
    (None, None),
)
ex_label = dataset_train.data_raw[i_test]
print(ex_sample)
ex_label

(('his', 273), (None, None))


{'id': 'train-7',
 'text': "Reb Chaim Yaakov's wife is the sister of Rabbi Moishe Sternbuch, as is the wife of Rabbi Meshulam Dovid Soloveitchik, making the two Rabbis his uncles. Reb Asher's brother Rabbi Shlomo Arieli is the author of a critical edition of the novallae of Rabbi Akiva Eiger. Before his marriage, Rabbi Arieli studied in the Ponevezh Yeshiva headed by Rabbi Shmuel Rozovsky, and he later studied under his father-in-law in the Mirrer Yeshiva.",
 'pron': 'his',
 'p_offset': 273,
 'entity_A': 'Reb Asher',
 'offset_A': 152,
 'is_coref_A': 'FALSE',
 'entity_B': 'Akiva Eiger',
 'offset_B': 253,
 'is_coref_B': 'FALSE'}

In [11]:
from collections import Counter
c = Counter()
for s in dataset_train:
    for e in s['entities']:
        c[e['is_coref']] += 1
c

Counter({'TRUE': 2684, 'FALSE': 3314})

In [12]:
global_params.update({ 
    'token_embeddings_len': len(dataset_train.tokenizer),
    'resize_token_embeddings': False, # ! not used
})

In [13]:
np.save(os.path.join(model_dir_path, 'global_params.npy'), global_params)

# Dataloader

In [14]:
from torch.utils.data import DataLoader

In [15]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
num_workers = 4

In [16]:
dataloader_train = DataLoader(
    dataset_train,
    batch_size=global_params['batch_size'],
    collate_fn=dataset_train.create_collate_fn(),
    num_workers=num_workers,
    shuffle=True,
)

dataloader_dev = DataLoader(
    dataset_dev,
    batch_size=global_params['batch_size'],
    collate_fn=dataset_dev.create_collate_fn(),
    num_workers=num_workers,
    shuffle=True,
)

In [17]:
for e in dataloader_dev:
    ex_in = e
    break

In [18]:
ex_in.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'words_ids', 'binary_is_coref'])

In [19]:
print(ex_in['input_ids'][0], ex_in['input_ids'].shape)
print(ex_in['words_ids'][0])
print(ex_in['binary_is_coref'][0])

tensor([  101,  7085,  9724,  1358,   146,  7831,  1161,   113,  1983,   117,
        10978,  2349,   114,   117, 14680,  6385,  1186, 19569, 21571,  9552,
          113,  1983,   117, 10978,  2349,   146,   118,  2684,   114,   117,
        10785, 23294,  2155, 16140,  9800,  2047,   113,  1983,   117, 10978,
         2349,   146,   118,  2684,  1957,  2596,   114,   117,  1847,  2091,
        18824,   113,  1483,   117, 10978,  2349,   114,  1105,  3270,  4409,
          113,  1483,   117, 10978,  2349,   146,   118,  2684,   114,  6114,
         6750,  2723,   113,  1145,  2752,  1106,  1112,  1615,  6750,  2723,
          114,  1110,  1141,  1104,  1103,  2746,  4245,   112,   188,  2020,
         4525,  2300, 12392,   119,  1124,  1110,  3025,  1510,   117,  1133,
         1674,  1136,  1294,  1251,  2357,  3178,  1235,  1103,  2985,  1544,
         1104,  1103,  1326,   119,   102,  1124,   117,  3270,  4409,   102,
            0,     0,     0,     0,     0,     0,     0,     0, 

# Printing model

In [20]:
from stud.modelsTests.utils.print_infos import print_summary, display_history, plot_confusion_matrix, print_classification_report

# Fine-tuning the model

In [21]:
from stud.modelsTests.model_3.model3_transformer_simple import Model3
import torch.optim as optim

loss_function = torch.nn.BCELoss()

final_model = Model3(
    device = device,
    loss_fn = loss_function,
    model_load_weights = False,
    fine_tune_transformer = False,
)

optimizer = optim.SGD(final_model.model.parameters(), lr=0.0016, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=0.0016)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
print_summary(final_model.model)

Model3_net(
  (transformer_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [23]:
history = {}

In [24]:
# import re

# position = 522
# text = "As Emperor Taizong of Tang wanted to enter into an alliance with Xueyantuo against Eastern Tujue, he sent the general Qiao Shiwang (***) as an envoy to Yi'nan, recognizing him the Zhenzhupiqie Khan (or Zhenzhu Khan in short), and awarding him with drums and banners. Yi'nan was very pleased, and he offered tribute to Emperor Taizong. It was said that by this point, <P>his</P> territory stretched from the Mohe to the east, Western Tujue to the west, and Gobi Desert to the south, and that many tribes, including Huige, Bayegu, Adie (**), Tongluo (**), Pugu (**), and Xi (*), all submitted to him."
# word = 'Adie (**'
# regex_subs = r"" + re.escape('<X>') + word + re.escape('</X>')
# re.sub(re.escape(word), regex_subs, text[position:],1)

In [25]:
ex_in_simple = dataset_train.create_collate_fn()([dataset_train[0], dataset_train[1]])
predictions = final_model.model(
    input_ids = ex_in_simple['input_ids'].to(device), 
    attention_mask = ex_in_simple['attention_mask'].to(device),
    token_type_ids = ex_in_simple['token_type_ids'].to(device),
)
labels = ex_in_simple['binary_is_coref'].to(device)

print(labels)
print(predictions)
print(final_model.model.compute_loss(predictions, labels ))

tensor([[1.],
        [0.]], device='cuda:0')
tensor([[0.3642],
        [0.3649]], device='cuda:0', grad_fn=<SigmoidBackward>)
tensor(0.7320, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>)


In [26]:
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
print(final_model.predict(dataset_train.data_raw[0]))

(('her', 274), (None, None))


In [27]:
from stud.modelsTests.utils.Trainer_model3_transformer_simple import Trainer_model3_transformer_simple

trainer = Trainer_model3_transformer_simple()

history = trainer.train(
    final_model, optimizer, dataloader_train, dataloader_dev,
    epochs=60, device=device,
    save_best=True, 
    min_score=0.7,
    save_path_name=os.path.join(model_dir_path, 'model3_weights_transformer_simple.pth'),
    saved_history=history
)

Epoch   0 => avg_loss: 0.701327
# Validation loss => 0.687788 | accuracy: 0.136564 #
Epoch   1 => avg_loss: 0.692595
# Validation loss => 0.694436 | accuracy: 0.462555 #
Epoch   2 => avg_loss: 0.696323
# Validation loss => 0.682222 | accuracy: 0.136564 #
Epoch   3 => avg_loss: 0.689592
# Validation loss => 0.681736 | accuracy: 0.143172 #
Epoch   4 => avg_loss: 0.695119
# Validation loss => 0.689663 | accuracy: 0.136564 #
Epoch   5 => avg_loss: 0.690969
# Validation loss => 0.695474 | accuracy: 0.436123 #
Epoch   6 => avg_loss: 0.688410
# Validation loss => 0.691392 | accuracy: 0.411894 #
Epoch   7 => avg_loss: 0.690299
# Validation loss => 0.683197 | accuracy: 0.138767 #
Epoch   8 => avg_loss: 0.691127
# Validation loss => 0.680749 | accuracy: 0.136564 #
Epoch   9 => avg_loss: 0.690062
# Validation loss => 0.681750 | accuracy: 0.136564 #
Epoch  10 => avg_loss: 0.690424
# Validation loss => 0.683250 | accuracy: 0.136564 #
Epoch  11 => avg_loss: 0.688831
# Validation loss => 0.679579 | a

KeyboardInterrupt: 